In [1]:
import dataset
import albumentations as A
import matplotlib.pyplot as plt
import pandas as pd
import torch

In [2]:
torch.manual_seed(0)

In [3]:
# MERGE TRAIN AND VAL
import pandas as pd

train = pd.read_csv('data/train.csv')
val = pd.read_csv('data/val.csv')
tPath = list(train.ImageID)
vPath = list(val.ImageID)
tPathOut = ['train/'+str(i) for i in tPath]
vPathOut = ['val/'+str(i) for i in vPath]
tPathOut.extend(vPathOut)
labels = list(train.label)
labels.extend(list(val.label))
train_ = pd.DataFrame({'ImageID':tPathOut,'label':labels})
train_.to_csv('data/trainAndVal.csv',index = False)

In [ ]:
import model
import albumentations as A
from albumentations.augmentations.transforms import Flip
import pytorch_lightning as pl
from pytorch_lightning import Trainer

img_size = 230

if __name__ == '__main__':
    ckpt = pl.callbacks.model_checkpoint.ModelCheckpoint(monitor = 'val_mse',save_top_k = 1,mode = 'min')
    trainer = Trainer(max_epochs = 50,gpus = 1, callbacks = ckpt, precision=16, amp_level='O1',deterministic=True,fast_dev_run = False)
    
    train_tr = A.Compose([
        A.CenterCrop(img_size,img_size,always_apply=True),
        Flip(p=0.25)
    ])
    
    val_tr = A.Compose([
        A.CenterCrop(img_size,img_size,always_apply=True)
    ])
    
    model = model.Classifier({'lr':3e-4,'batch_size':48,'train_tr':train_tr,'val_tr':val_tr})
    
    #ckpt = torch.load(path)
    #model.load_state_dict(ckpt['state_dict'])
    trainer.fit(model)
    trainer.test(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


Loaded pretrained weights for efficientnet-b4



  | Name   | Type         | Params
----------------------------------------
0 | resnet | EfficientNet | 18.2 M
----------------------------------------
18.2 M    Trainable params
0         Non-trainable params
18.2 M    Total params
72.776    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [3]:
import model
import torch
import albumentations as A
from albumentations.augmentations.transforms import Flip, VerticalFlip, HorizontalFlip
import pytorch_lightning as pl
from pytorch_lightning import Trainer
m = 0

tr0 = A.Compose([
        A.CenterCrop(230,230,always_apply=True)
])

tr1 = A.Compose([
    A.CenterCrop(230,230,always_apply=True),
    A.VerticalFlip(p = 1)
])

tr2 = A.Compose([
    A.CenterCrop(230,230,always_apply=True),
    A.HorizontalFlip(p = 1)
])

tr3 = A.Compose([
    A.CenterCrop(230,230,always_apply=True),
    A.HorizontalFlip(p = 1),
    A.VerticalFlip(p = 1)
])

tr_list = [tr0,tr1,tr2,tr3]
concat = torch.tensor([])

for tr in tr_list:
    if __name__ == '__main__':

        path = '' #PATH OF LAST CHECKPOINT
        trainer = Trainer(gpus = 1, precision=16, amp_level='O1',deterministic=True)

        train_tr = A.Compose([
            A.CenterCrop(230,230,always_apply=True)
        ])

        val_tr = A.Compose([
            A.CenterCrop(230,230,always_apply=True)
        ])

        m = model.Classifier({'lr':3e-4,'batch_size':48,'train_tr':train_tr,'val_tr':tr})

        ckpt = torch.load(path)
        m.load_state_dict(ckpt['state_dict'])

        out = trainer.predict(m)
        output = torch.tensor([])

        for i in range(len(out)):
            output = torch.cat((output,torch.tensor(out[i]).argmax(1)))
        concat = torch.cat((concat,output.unsqueeze(0)),0)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


Loaded pretrained weights for efficientnet-b4


Predicting: 0it [00:00, ?it/s]

In [62]:
output = torch.round(concat.mean(0))

In [31]:
import model
model.writeSub(output) 